In [ ]:
# default_exp reward

# Reward

> Reward function

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.chem import *
from mrl.templates import *
from mrl.agent import *
from mrl.environment import *

In [ ]:
# export

class Reward(Callback):
    def __init__(self, name, order=10, weight=1., track=True):
        self.name = name
        self.order = order
        self.track = track
        self.weight = weight
        
    def setup(self):
        log = self.environment.log
        log.add_log(self.name)
        if self.track:
            log.add_metric(self.name)
            
    def _compute_reward(self):
        raise NotImplementedError
    
    def compute_reward(self):
        rewards = self._compute_reward()
        rewards = rewards.squeeze()
        self.batch_state.rewards += self.weight*rewards
        self.batch_state[self.name] = rewards
        
        if self.track:
            self.environment.log.update_metric(self.name, rewards.mean().detach().cpu().numpy())
            
            
class FunctionReward(Reward):
    def __init__(self, reward_function, name, order=10, weight=1., track=True):
        super().__init__(name, order, weight, track)
        self.reward_function = reward_function
        
    def _compute_reward(self):
        return self.reward_function(self.batch_state)
        
        
class SampleReward(Reward):
    def __init__(self, reward_function, template_filter, lookup, 
                 name, order=10, weight=1., track=True):
        super().__init__(name, order, weight, track)
        self.reward_function = reward_function
        self.lookup = lookup
        self.template_filter = template_filter
        self.lookup_table = {}
    
    def _compute_reward(self):
        
        samples = self.batch_state.samples
        hps = self.batch_state.template_passes
        outputs = to_device(torch.tensor([0. for i in samples]))
        
        to_score = []
        to_score_idxs = []
        
        for i, sample in enumerate(samples):
            if self.lookup and sample in self.lookup_table.keys():
                outputs[i] = self.lookup_table[sample]
                
            else:
                if (self.template_filter and hps[i]) or (not self.template_filter):
                    to_score.append(sample)
                    to_score_idxs.append(i)
                    
        if to_score:
            scores = self.reward_function(samples)
            
            for i in range(len(to_score)):
                outputs[to_score_idxs[i]] = scores[i]
            
                if self.lookup:
                    item_score = scores[i]
                    if type(item_score) == torch.Tensor:
                        item_score = item_score.detach().cpu()
                    self.lookup_table[to_score[i]] = item_score
        
        
        return outputs
    

In [ ]:
# export

class NoveltyBonus(Reward):
    def __init__(self, weight, name='novel', order=100, track=True):
        super().__init__(name, order, weight, track)
        
    def _compute_reward(self):
        log = self.environment.log
        state = self.batch_state
        old = log.unique_samples
        
        new = [not i in old for i in state.samples]
        reward = to_device(torch.tensor(new)).float()
        return reward


parallel reward
    parallel process calculation on one sequence at a time
    
batch reward
    parallel featurize
    batch
    compute